<img src="img/cabecera.png?raw=1">

# Taller: **Construye un asistente de IA con Streamlit y Replicate**

*Rodrigo Oliver*  
*Lead Instructor Bootcamp Data Science Online*

### Requisitos previos Parte 2 

- Una cuenta en [GitHub](https://github.com/signup)
- Una cuenta en [Replicate](https://replicate.com) para obtener un API Token
- Python 3.8 o superior (si vas a hacer el taller en local)

## Parte 2: Interfaz conversacional con Streamlit  

Adaptado y actualizado de [*streamlit-replicate-app*](https://github.com/sfc-gh-cnantasenamat/streamlit-replicate-app)
y de [*llama2-chatbot*](https://github.com/a16z-infra/llama2-chatbot)

### **¿Qué es Streamlit?**

Streamlit es un framework gratuito y de código abierto para construir y compartir rápidamente aplicaciones web de Ciencia de Datos. Es una biblioteca basada en Python y diseñada específicamente pensando en profesionales del dato.

### **¿Por qué Streamlit?**

Sencillamente, porque los/las profesionales de la Ciencia de Datos (Data Analysts, Data Scientists, ML Engineers, AI Engineers, etc.) no somos Desarrolladores Web.  

Aunque podamos estar familiarizados con otros lenguajes o stacks tecnológicos (muy recomendable para poder comunicarse eficazmente con otros profesionales), nuestro objetivo no es dominar las herramientas necesarias para crear aplicaciones web.  

Sin embargo, eso no significa que nos tengamos que conformar con exponer nuestros proyectos en cuadernos Jupyter. Streamlit nos permite crear aplicaciones de datos con un aspecto muy profesional con unas pocas líneas de código, sin tener que depender de la ayuda de otros compañeros o departamentos.  



### Instalación
Si has llegado hasta aquí, lo más probable es que:
- Hayas utilizado el repositorio del taller como plantilla
- Estés viendo este cuaderno en Codespaces de GitHub

En tal caso, Codespaces ya se habrá ocupado de instalar las dependencias necesarias para el proyecto que se encuentran en el archivo `requirements.txt`

Si prefieres realizar el taller en local, te recomendamos crear un entorno virtual e instalar las dependencias necesarias para el desarrollo del proyecto.  

Los directorios y archivos contienen la estructura inicial básica del proyecto, incluyendo un directorio `.streamlit` con los archivos `toml` que definen la configuración mínima de nuestra app.  

[**Si no sabes qué es un TOML, pulsa aquí**](https://es.wikipedia.org/wiki/TOML)

### Ejecución de la aplicación

En cuanto tu script de Python tenga la configuración inicial podrás ejecutar tu aplicación desde el terminal con:

```bash
python -m streamlit run chatbot.py
```

## **Paso 1: Configuración inicial**

Una vez completada la fase previa de instalación, abre el archivo `chatbot.py` en Codespaces o en tu IDE favorito.  

Verás que ya contiene la configuración inicial. Pero si prefieres trastear también puedes crearlo de cero con el siguiente código:

In [ ]:
import streamlit as st
import replicate
import os

# Código refactorizado de https://github.com/sfc-gh-cnantasenamat/streamlit-replicate-app
# y https://github.com/a16z-infra/llama2-chatbot

# Configuración del proyecto
APP_NAME = 'My First Chatbot'
MODEL_NAME = 'Meta Llama 3 8B Instruct'
MODEL_ENDPOINT = 'meta/meta-llama-3-8b-instruct'
MODEL_DOC_LINK = 'https://replicate.com/meta/meta-llama-3-8b-instruct'

# Configuración inicial
st.set_page_config(
    page_title=APP_NAME, 
    page_icon=':robot:'
    )

## **Paso 2: Inicialización de variables en el estado de la sesión**

Streamlit utiliza un estado de sesión para mantener variables entre ejecuciones. Inicializamos las variables que necesitaremos:

[**Si quieres aprender más sobre Session State en Streamlit, pulsa aquí**](https://docs.streamlit.io/develop/api-reference/caching-and-state/st.session_state)

In [ ]:
# Inicialización de variables
default_system_prompt = '''You are a helpful assistant.
You do not respond as "User" or pretend to be "User".
You only respond once as "Assistant".'''

default_start_message = 'How may I assist you today?'

if 'messages' not in st.session_state:
    st.session_state.messages = [{'role': 'assistant', 'content': default_start_message}]
if 'system_prompt' not in st.session_state:
    st.session_state.system_prompt = default_system_prompt

## **Paso 3: Barra lateral - Estructura básica y API token**

Parece que todos nuestros chatbots favoritos dividen su interfaz en una barra lateral y un espacio principal de conversación, así que podemos inspirarnos en ellos.

Streamlit nos permite construir barras laterales con facilidad, comencemos con su estructura básica donde incluiremos la configuración del API token:

In [ ]:
# Barra lateral
with st.sidebar:
    st.title(APP_NAME)
    
    # API token
    try:
        if st.secrets and 'REPLICATE_API_TOKEN' in st.secrets:
            replicate_api_token = st.secrets['REPLICATE_API_TOKEN']
            st.success('API token already provided!', icon='✅')
        else:
            raise KeyError
    except:
        replicate_api_token = st.text_input('Enter Replicate API token:', type='password')
        if not (replicate_api_token and replicate_api_token.startswith('r8_') and len(replicate_api_token) == 40):
            st.warning('Please enter a valid Replicate API token!', icon='⚠️')
            if replicate_api_token:
                st.info("Replicate tokens start with 'r8_' and are 40 characters long")
        else:
            st.success('Proceed to chat!', icon='👉')
    
    if replicate_api_token:
        os.environ['REPLICATE_API_TOKEN'] = replicate_api_token


A partir de este punto para ejecutar la aplicación es recomendable crear un archivo `secrets.toml` dentro del directorio `.streamlit/`con tu API key de Replicate.

Puedes aprovechar el archivo de ejemplo `secrets.example.toml` como base, en su interior verás

```toml
REPLICATE_API_TOKEN = "tu_api_token_de_replicate"
```

Esto nos permitirá testear la aplicación durante su desarrollo, pero recuerda que **nunca debes subir tus secretos a GitHub**.  

Cuando despleguemos nuestro chatbot podrás decidir si facilitamos una API token (y asumimos los gastos del chatbot) o dejamos que los usuarios utilicen sus propios API token.

## **Paso 4: Función para generar respuestas**

De momento dejaremos la barra lateral así, porque queremos chatear con nuestro asistente. 

Para ello definimos la función que se comunicará con Replicate para obtener respuestas del modelo. Esta función es el corazón de nuestro chatbot y realiza varias tareas clave:

1. **Formatea el historial de conversación**:
   - Comienza con el System Prompt (instrucciones iniciales para el modelo) y tras ello agrega todo el historial de mensajes con formato "User:" y "Assistant:".
   - Esto es importante porque permite que el modelo tenga contexto completo de la conversación.

2. **Configura los parámetros de generación**:
   - Prepara los parámetros que controlarán el comportamiento del modelo, de momento solo incluiremos el prompt pero recuerda que puedes ajustar parámetros como temperatura, longitud máxima, penalización por repetición, etc.

3. **Transmite la respuesta en tiempo real**:
   - Utiliza la función `stream` de Replicate para obtener tokens de respuesta gradualmente e implementa un **generador** que permite mostrar la respuesta token por token.
   - Esto crea una experiencia más natural donde el usuario ve la respuesta formándose progresivamente.

In [ ]:
# Generación de respuestas
def generate_response():
    conversation_context = f'System: {st.session_state.system_prompt}\n\n'
    for dict_message in st.session_state.messages:
        role = dict_message.get('role', '')
        if role in ('user', 'assistant'):
            conversation_context += f'{role.capitalize()}: {dict_message["content"]}\n\n'
    
    input_params = {
        'prompt': f'{conversation_context}Assistant: '
        }
    
    try:
        for event in replicate.stream(MODEL_ENDPOINT, input=input_params):
            yield str(event)
    except replicate.exceptions.ReplicateError as e:
        yield f'Replicate API Error: {e}'
    except Exception as e:
        yield f'Unexpected error: {e}. Please check your connection and API token.'


## **Paso 5: Interfaz de chat**

Ahora que tenemos lista la función principal de nuestro asistente, creamos la interfaz de chat que mostrará los mensajes y permitirá al usuario interactuar con el chatbot:

In [ ]:
# Entrada del usuario
if prompt := st.chat_input('Type your message here...', disabled=not replicate_api_token):
    st.session_state.messages.append({'role': 'user', 'content': prompt})

# Mostrar mensajes
for message in st.session_state.messages:
    with st.chat_message(message['role']):
        st.write(message['content'])

# Generar respuesta
if st.session_state.messages and st.session_state.messages[-1]['role'] == 'user':
    with st.chat_message('assistant'):
        with st.spinner('Thinking...'):
            response = generate_response()
            full_response = st.write_stream(response)
    st.session_state.messages.append({'role': 'assistant', 'content': full_response})

Técnicamente, llegados a este punto el código es suficiente para hacer funcionar nuestro chatbot, pero creo que le vendrá bien algo más de funcionalidad.

## **Paso 6: Barra lateral - System Prompt Editable**

Dado que hemos hecho tanto hincapié en la importancia del System Prompt, ¿por qué no añadir un área de texto con la que poder hacer pruebas más fácilmente?

Adelante, añadimos el área de texto para editar el prompt de sistema y así poder definir el comportamiento de nuestro chatbot.

Importante, para que quede más integrado en la interfaz de nuestra app vamos a incluir este código en la barra lateral. Busca su posición en el archivo base (guíate por los comentarios) y ten cuidado con las indentaciones:

In [ ]:
    # System Prompt editable
    st.subheader('System Prompt')

    def on_system_prompt_change():
        st.session_state.system_prompt = st.session_state.system_prompt_textarea

    st.text_area(
        'Edit the prompt that guides the model:',
        value=st.session_state.system_prompt,
        height=150,
        key='system_prompt_textarea',
        on_change=on_system_prompt_change
    )

## **Paso 7: Barra lateral - Información del modelo**

Parece que la aplicación empieza a tener buena pinta, ¿qué os parece si añadimos algo de información sobre nuestro querido modelo?.

> Recuerda que es totalmente opcional, pero quizá esto te haga reflexionar sobre qué pasaría si, por ejemplo, quisieramos incluir un desplegable para cambiar el modelo base a nuestro gusto.
>
> Te animamos a que lo intentes y hagas tus propios cambios en el código (después de terminar el taller).  

Recuerda que va en la barra lateral, cuidado con las indentaciones:

In [ ]:
    # Información del modelo
    st.subheader('Model')
    st.info(f'Using {MODEL_NAME}')
    st.markdown(f'👉 [Learn more about this model]({MODEL_DOC_LINK}) 👈')

## **Paso 8: Barra lateral - Botón de limpiar historial**

Para terminar nuestro proyecto, vamos a implementar un botón para limpiar el historial de chat, esto borrará los mensajes del estado de sesión de modo que "reiniciará" la conversación de forma similar a cuando iniciamos un nuevo chat en ChatGPT.

Recuerda que es parte de la barra lateral así que cuidado con las indentaciones:

In [ ]:
    # Botón para limpiar historial 
    def clear_chat_history():
        st.session_state.messages = [{'role': 'assistant', 'content': default_start_message}]

    st.button('Clear Chat', on_click=clear_chat_history, use_container_width=True)


Y ahora sí, nuestro proyecto está listo para desplegar en la nube.

## **Paso 9: Configuración para despliegue**

Para desplegar tu aplicación en Streamlit Cloud:

1. Haz commit de los cambios desde Codespaces (o desde tu equipo si estás trabajando el local) a tu repositorio de GitHub, para ello ve al terminal y ejecuta los siguientes comandos:

    ```bash
    git add .
    git commit -m "Código chatbot"
    git push
    ```

2. Conéctate a [Streamlit Cloud](https://streamlit.io/cloud) y selecciona la versión Free. Date de alta usando tu cuenta de GitHub.
3. Selecciona tu repositorio, la rama y el archivo principal.
4. Configura tus secretos en la interfaz de Streamlit Cloud, solo si quieres y bajo tu responsabilidad (puedes acceder desde el menú de tres puntos de tu aplicación).
5. Es hora de desplegar tu aplicación.

# **¡Felicidades!**  

Has creado y desplegado un chatbot básico con Streamlit y Replicate.

# **Muy Importante**

**Recuerda detener tu entorno de Codespaces** para no incurrir en posibles gastos de facturación. Si te deja más tranquil@ también puedes eliminar todo el espacio de desarrollo y volver a levantarlo de nuevo más adelante.

# **Gracias por vuestra atención :)**